In [1]:
import convml_tt
import convml_tt.utils
from convml_tt.architectures.triplet_trainer import TileType
import convml_tt.data.dataset

from pathlib import Path
import matplotlib.pyplot as plt

import xarray as xr
import seaborn as sns

In [2]:
convml_tt.__version__

'0.4.0'

In [3]:
import shapely.geometry as geom
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pathlib import Path
import yaml

In [4]:
tt_ds = convml_tt.data.dataset.TripletDataset.load('/home/earlcd/Desktop/mldev/')
tt_ds

/home/earlcd/pytools/convml_tt/data/dataset/__init__.py:48: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(fh.read())


{'N_days': 1,
 'N_hours_from_zenith': 0.25,
 'N_triplets': {'study': 0, 'train': 0},
 'channels': [1, 2, 3],
 'data_path': PosixPath('/home/earlcd/Desktop/mldev'),
 'domain_bbox': [[-60, 10], [-30, 30]],
 'name': 'mldev',
 't_start': datetime.datetime(2020, 1, 18, 0, 0),
 'tile_N': 256,
 'tile_size': 200000.0}

In [5]:
tt_ds.get_dataset_scene(scene_num=0,)

/home/earlcd/anaconda2/envs/convml_tt/lib/python3.7/site-packages/luigi/parameter.py:283: UserWarning: Parameter "data_path" with value "/home/earlcd/Desktop/mldev" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/earlcd/pytools/convml_tt/data/sources/satellite/pipeline.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(fh)
DEBUG: Checking if CreateRGBScene(source_fns=["ABI-L1b-RadF/2020/018/13/OR_ABI-L1b-RadF-M6C01_G16_s20200181340195_e20200181349503_c20200181349555.nc", "ABI-L1b-RadF/2020/018/13/OR_ABI-L1b-RadF-M6C02_G16_s20200181340195_e20200181349503_c20200181349551.nc", "ABI-L1b-RadF/2020/018/13/OR_ABI-L1b-RadF-M6C03_G16_s20200181340195_e20200181349503_c20200181349562.nc"], domain_bbox=[[-60, 10], [-30, 30]], domain_bbox_pad_frac=0.05, data_path=/home/earlcd/Desk

Exception: Task didn't complete

In [ ]:
#from convective_organisation.data.processing import goes16_oldcomp_500
#from convective_organisation.data.sources.satdata.bbox import LatLonBox
#from convective_organisation.data.sources.satdata import processing
#from convective_organisation.data.sources import satdata

In [ ]:
data_path = Path('../../data/storage')

In [ ]:
tileset_path = data_path/"tiles"/"goes16"/"Nx256_s200000.0_N1000study_N10000train"
tile_path = tileset_path/"train"

In [ ]:
triplets = convml_tt.NPMultiImageList.from_folder(tile_path)

In [ ]:
#### 
m = 194
triplets[m][0].show(), triplets[m][1].show(), triplets[m][2].show()

In [ ]:
import convml_tt.interpretation.tile_aggregation

In [ ]:
tiles = list(zip(*[
    load_tile_definitions(triplets[m:m+1], tile_type=tile_type)
    for tile_type in [TileType.ANCHOR, TileType.NEIGHBOR, TileType.DISTANT]
]))[0]

In [ ]:
tiles

In [ ]:
tile_a, tile_n, tile_d = tiles

tile_a, tile_n = tile_n, tile_a

In [ ]:
scene_fns = tile_a.meta['rgb_source_files']

In [ ]:
TILING_BBOX_EXTENT = [
    (-60, 10), # (lon, lat) min
    (-30, 30) # (lon, lat) max
]

In [ ]:
cli = satdata.Goes16AWS()
path_composites = data_path/"composites"
da_rgb_composite = satdata.processing.load_data_for_rgb([scene_fns, ], cli=cli, path_composites=path_composites,
                                                bbox_extent=TILING_BBOX_EXTENT,
                                                use_old=False)[0]

In [ ]:
scene_fns = tile_d.meta['rgb_source_files']

da_rgb_composite_dist = satdata.processing.load_data_for_rgb([scene_fns, ], cli=cli, path_composites=path_composites,
                                                bbox_extent=TILING_BBOX_EXTENT,
                                                use_old=False)[0]

In [ ]:
from convml_tt.data.sources.satdata.bbox import LatLonBox
from convml_tt.data.sources.satdata.satpy_rgb import rgb_da_to_img

tiling_bbox = LatLonBox(TILING_BBOX_EXTENT)

In [ ]:
bbox_shape = tiling_bbox.get_outline_shape()

s = 5
img_rgb = rgb_da_to_img(da_rgb_composite.isel(x=slice(None, None, s), y=slice(None, None, s)))

In [ ]:
fig = plt.figure(figsize=(8, 6))
#projection = ccrs.Geostationary(central_longitude=-60) # too costly when we're showing the composite
projection = da_rgb_composite.crs
ax = plt.subplot(projection=projection)

#da_rgb_composite.sel(bands='R').isel(x=slice(None, None, 100), y=slice(None, None, 100)).plot(
#ax=ax, transform=da_rgb_composite.crs, rasterized=True
#)
#da_rgb_composite.sel(bands='R').plot(ax=ax, transform=da_rgb_composite.crs, rasterized=True)

img_rgb.data.plot.imshow(
    ax=ax, transform=da_rgb_composite.crs, rasterized=True, rgb='bands'
)

gl = ax.gridlines(linestyle='--', draw_labels=False)
ax.coastlines(resolution='10m', color='grey')

def draw_box(geom, color, face_alpha=0.5):

    ax.add_geometries([geom,], crs=ccrs.PlateCarree(), alpha=face_alpha,
              edgecolor=color, facecolor=color)
    ax.add_geometries([geom,], crs=ccrs.PlateCarree(), alpha=face_alpha*2.0,
              edgecolor=color, facecolor='none', linewidth=1.0)
    
draw_box(bbox_shape, color='red', face_alpha=0.2)
draw_box(tile_a.get_outline_shape(), color='green')
draw_box(tile_n.get_outline_shape(), color='blue')

ax.set_extent([-70, -10, 5, 35], crs=ccrs.PlateCarree())  # [x0, x1, y0, y1]

plt.savefig('domain_overview_triplets.pdf')

In [ ]:
rgb_tile_a = tile_a.crop_field(da_rgb_composite)

In [ ]:
tile_a.create_true_color_img(da_rgb_composite, resampling_N=256)

In [ ]:
tile_n.create_true_color_img(da_rgb_composite, resampling_N=256)

In [ ]:
tile_d.create_true_color_img(da_rgb_composite_dist, resampling_N=256)